In [1]:
## Standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import pandas 
from datetime import datetime
import seaborn as sns
import glob
from os.path import basename
sns.set_context('paper')
import re
import csv
import json
import time
from scipy import stats
import numpy as np

## Updated code to compare single samples with increased terms, and bug fix for Tau

In [2]:
topics = glob.glob('../data/sampling-100k/*')

In [3]:
for t in topics:
    print(t)

../data/sampling-100k/climate_change
../data/sampling-100k/community_policing
../data/sampling-100k/deep_state
../data/sampling-100k/ebola
../data/sampling-100k/gun_violence
../data/sampling-100k/immigration
../data/sampling-100k/network_neutrality
../data/sampling-100k/teen_pregnancy
../data/sampling-100k/us_election
../data/sampling-100k/vaccines


## First load and count words

In [18]:
for t in topics:
    topic_name = basename(t)
    print(t)
    # load data
    one_k = pandas.read_csv(t + '/sample1000.csv', index_col=0)
    ten_k = pandas.read_csv(t + '/sample10000.csv', index_col=0)
    hundred_k = pandas.read_csv(t + '/sample100000.csv', index_col=0)
    
    
    # drop if empty
    if (one_k.empty or ten_k.empty or hundred_k.empty):
        print("empty")
        continue
        
        
    one_k = one_k.dropna()
    ten_k = ten_k.dropna()
    hundred_k = hundred_k.dropna()
        
    
    print("Number of unique stems in 1K:", len(one_k.stem.unique()))
    print("Number of unique stems in 10K:", len(ten_k.stem.unique()))
    print("Number of unique stems in 100K:", len(hundred_k.stem.unique()))
    print("") 

../data/sampling-100k/climate_change
Number of unique stems in 1K: 2733
Number of unique stems in 10K: 11432
Number of unique stems in 100K: 33739

../data/sampling-100k/community_policing
empty
../data/sampling-100k/deep_state
Number of unique stems in 1K: 2532
Number of unique stems in 10K: 12582
Number of unique stems in 100K: 44161

../data/sampling-100k/ebola
Number of unique stems in 1K: 2463
Number of unique stems in 10K: 10906
Number of unique stems in 100K: 39754

../data/sampling-100k/gun_violence
Number of unique stems in 1K: 2887
Number of unique stems in 10K: 12526
Number of unique stems in 100K: 40741

../data/sampling-100k/immigration
Number of unique stems in 1K: 2789
Number of unique stems in 10K: 11536
Number of unique stems in 100K: 39041

../data/sampling-100k/network_neutrality
Number of unique stems in 1K: 1979
Number of unique stems in 10K: 8650
Number of unique stems in 100K: 25957

../data/sampling-100k/teen_pregnancy
Number of unique stems in 1K: 2127
Number o

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of unique stems in 1K: 2545
Number of unique stems in 10K: 11814
Number of unique stems in 100K: 42890



## Test weighted Tau

In [22]:
from scipy.stats import weightedtau

Compare two of the same

In [24]:
weightedtau([1,2,3,4,5,6,7,8],[[1,2,3,4,5,6,7,8]])

WeightedTauResult(correlation=0.99999999999999978, pvalue=nan)

Swap high-rank (6 and 8)

In [25]:
weightedtau([1,2,3,4,5,6,7,8],[[1,2,3,4,5,8,7,6]])

WeightedTauResult(correlation=0.61454226894437147, pvalue=nan)

Swap low rank (1 and 3, 2 and 3)


In [26]:
weightedtau([1,2,3,4,5,6,7,8],[[3,2,1,4,5,8,7,6]])

WeightedTauResult(correlation=0.52318378073962835, pvalue=nan)

In [27]:
 weightedtau([1,2,3,4,5,6,7,8],[[1,3,2,4,5,8,7,6]])

WeightedTauResult(correlation=0.58200362930980543, pvalue=nan)

## Correlating counts

In [53]:
from scipy.stats import pearsonr

In [64]:
for t in topics:
    topic_name = basename(t)
    print(topic_name)
    # load data
    one_k = pandas.read_csv(t + '/sample1000.csv', index_col=0)
    ten_k = pandas.read_csv(t + '/sample10000.csv', index_col=0)
    hundred_k = pandas.read_csv(t + '/sample100000.csv', index_col=0)
    
    
    # drop if empty
    if (one_k.empty or ten_k.empty or hundred_k.empty):
        print("empty")
        continue
        
        
    one_k = one_k.dropna()
    ten_k = ten_k.dropna()
    hundred_k = hundred_k.dropna()
        
    hundred_k.sort_values('count', ascending=False)[:100]

    hundred_k_counts = hundred_k['count'][:100]
    one_k_counts = one_k[one_k.index.isin(hundred_k.index[:100])]
    ten_k_counts = ten_k[ten_k.index.isin(hundred_k.index[:100])]
    
    print("Pearson correlation, 1K:100K", pearsonr(one_k_counts['count'].values, hundred_k_counts.values))
    print("Pearson correlation, 10K:100K", pearsonr(ten_k_counts['count'].values, hundred_k_counts.values))

    print()

climate_change
Pearson correlation, 1K:100K (0.9998725206911947, 6.6211289831971078e-178)
Pearson correlation, 10K:100K (0.99997488752900077, 1.7783424008115234e-212)

community_policing
empty
deep_state
Pearson correlation, 1K:100K (0.9861667371768581, 2.6255632935354593e-78)
Pearson correlation, 10K:100K (0.99939164417043047, 1.1835863280723839e-144)

ebola
Pearson correlation, 1K:100K (0.99969572682265739, 2.1503070970824527e-159)
Pearson correlation, 10K:100K (0.99990543531029874, 2.9204115872097447e-184)

gun_violence
Pearson correlation, 1K:100K (0.99864713946268346, 1.1837606733509528e-127)
Pearson correlation, 10K:100K (0.99926777830971913, 1.0368004001333353e-140)

immigration
Pearson correlation, 1K:100K (0.99969308668636103, 3.2834044806212231e-159)
Pearson correlation, 10K:100K (0.99987259650241622, 6.4309288905145386e-178)

network_neutrality
Pearson correlation, 1K:100K (0.9986271338848639, 2.4290817547644429e-127)
Pearson correlation, 10K:100K (0.99995818643581347, 1.258

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
